In [4]:
import csv
import math

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import keras
from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import keras.backend as K

from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Concatenate, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

WEIGHTS_PATH = 'https://github.com/wohlert/keras-squeezenet/releases/download/v0.1/squeezenet_weights.h5'

def _fire(x, filters, name="fire"):
    sq_filters, ex1_filters, ex2_filters = filters
    squeeze = Convolution2D(sq_filters, (1, 1), activation='relu', padding='same', name=name + "squeeze1x1")(x)
    expand1 = Convolution2D(ex1_filters, (1, 1), activation='relu', padding='same', name=name + "expand1x1")(squeeze)
    expand2 = Convolution2D(ex2_filters, (3, 3), activation='relu', padding='same', name=name + "expand3x3")(squeeze)
    x = Concatenate(axis=-1, name=name)([expand1, expand2])
    return x

def SqueezeNet(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=1000):

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu", name='conv1')(img_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="valid")(x)

    x = _fire(x, (16, 64, 64), name="fire2")
    y = _fire(x, (16, 64, 64), name="fire3")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)
    
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="valid")(x)

    x = _fire(x, (32, 128, 128), name="fire4")
    y = _fire(x, (32, 128, 128), name="fire5")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)   

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="valid")(x)

    x = _fire(x, (48, 192, 192), name="fire6")
    y = _fire(x, (48, 192, 192), name="fire7")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)

    x = _fire(x, (64, 256, 256), name="fire8")
    y = _fire(x, (64, 256, 256), name="fire9")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)

    if include_top:
        x = Dropout(0.5, name='dropout9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = AveragePooling2D(pool_size=(13, 13), name='avgpool10')(x)
        x = Flatten(name='flatten10')(x)
        x = Activation("softmax", name='softmax')(x)
    else:
        if pooling == "avg":
            x = GlobalAveragePooling2D(name="avgpool10")(x)
        else:
            x = GlobalMaxPooling2D(name="maxpool10")(x)

    model = Model(img_input, x, name="squeezenet")

    if weights == 'imagenet':
        weights_path = get_file('squeezenet_weights.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models')

        model.load_weights(weights_path)

    return model

# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        label = row[3]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))
        
train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 4)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 4)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)
                # Convert to 1 hot vector
                p = prices[index]
                if p == "25":
                    Y[i,:] = np.array([1,0,0,0])
                if p == "50":
                    Y[i,:] = np.array([0,1,0,0])
                if p == "75":
                    Y[i,:] = np.array([0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            X = seq.augment_images(X)
            
            yield (X, Y)

# Hyperparameters

num_settings = 1

hp_dropout = [0.5] * num_settings

#RMSprop
hp_lr = [0.005] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [256] * num_settings

# Minibatch size
hp_mbsize = [256] * num_settings

num_epochs = 500


(19658,)
(2185,)


In [ ]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    model = SqueezeNet(include_top=True)
    
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Convolution2D(256, (1, 1), padding='valid', name='top_conv', input_shape=(model.layers[-1].output_shape[1:])))
    top_model.add(AveragePooling2D(pool_size=(5, 5), name='top_avgpool'))
    top_model.add(Flatten(input_shape=(model.layers[-1].output_shape[1:]),name='top_flatten'))
    top_model.add(Dropout(hp_dropout[setting], name='top_dropout'))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform', name='top_dense'))
    top_model.add(Dense(4, activation='softmax', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.layers[-1].output))
    new_model.summary()

    new_model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
    
    
    checkpoint_path = 'output/bikes-cnn-PriceNet-Class-Aug/{epoch:05d}.hdf5'
    
    # keep a checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path, period=5)
    
    
    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=num_epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps,
        callbacks=[checkpoint])
    
   
    print("==========")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 55, 55, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2squeeze1x1 (Conv2D)        (None, 55, 55, 16)   1040        maxpool1[0][0]                   
__________________________________________________________________________________________________
fire2expan

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:48: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:48: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=77, epochs=500, validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=9)`


Epoch 1/500
77/77 [==============================] - 212s 3s/step - loss: 1.2623 - acc: 0.4095 - val_loss: 1.1606 - val_acc: 0.4568
Epoch 2/500
77/77 [==============================] - 190s 2s/step - loss: 1.0754 - acc: 0.5021 - val_loss: 1.1437 - val_acc: 0.4715
Epoch 3/500
77/77 [==============================] - 188s 2s/step - loss: 0.9957 - acc: 0.5431 - val_loss: 1.0637 - val_acc: 0.5252
Epoch 4/500
77/77 [==============================] - 188s 2s/step - loss: 0.9484 - acc: 0.5691 - val_loss: 0.9477 - val_acc: 0.5510
Epoch 5/500
77/77 [==============================] - 188s 2s/step - loss: 0.9017 - acc: 0.5883 - val_loss: 0.9707 - val_acc: 0.5686
Epoch 6/500
77/77 [==============================] - 189s 2s/step - loss: 0.8857 - acc: 0.5943 - val_loss: 0.9344 - val_acc: 0.5989
Epoch 7/500
77/77 [==============================] - 189s 2s/step - loss: 0.8483 - acc: 0.6158 - val_loss: 0.8997 - val_acc: 0.5944
Epoch 8/500
77/77 [==============================] - 190s 2s/step - loss: 0.

77/77 [==============================] - 188s 2s/step - loss: 0.4542 - acc: 0.8072 - val_loss: 0.7189 - val_acc: 0.7083
Epoch 63/500
77/77 [==============================] - 188s 2s/step - loss: 0.4469 - acc: 0.8084 - val_loss: 0.5810 - val_acc: 0.7579
Epoch 64/500
77/77 [==============================] - 189s 2s/step - loss: 0.4518 - acc: 0.8060 - val_loss: 0.7332 - val_acc: 0.7071
Epoch 65/500
77/77 [==============================] - 188s 2s/step - loss: 0.4435 - acc: 0.8139 - val_loss: 0.7622 - val_acc: 0.6932
Epoch 66/500
77/77 [==============================] - 187s 2s/step - loss: 0.4397 - acc: 0.8098 - val_loss: 0.6070 - val_acc: 0.7571
Epoch 67/500
77/77 [==============================] - 185s 2s/step - loss: 0.4401 - acc: 0.8142 - val_loss: 0.5944 - val_acc: 0.7517
Epoch 68/500
77/77 [==============================] - 186s 2s/step - loss: 0.4358 - acc: 0.8139 - val_loss: 0.9630 - val_acc: 0.6583
Epoch 69/500
77/77 [==============================] - 187s 2s/step - loss: 0.4343 

77/77 [==============================] - 186s 2s/step - loss: 0.3043 - acc: 0.8753 - val_loss: 0.6611 - val_acc: 0.7501
Epoch 124/500
77/77 [==============================] - 187s 2s/step - loss: 0.3038 - acc: 0.8737 - val_loss: 0.6133 - val_acc: 0.7784
Epoch 125/500
77/77 [==============================] - 185s 2s/step - loss: 0.2971 - acc: 0.8752 - val_loss: 0.5832 - val_acc: 0.7816
Epoch 126/500
77/77 [==============================] - 186s 2s/step - loss: 0.2977 - acc: 0.8758 - val_loss: 0.9042 - val_acc: 0.6817
Epoch 127/500
77/77 [==============================] - 186s 2s/step - loss: 0.2955 - acc: 0.8766 - val_loss: 0.5769 - val_acc: 0.7886
Epoch 128/500
77/77 [==============================] - 187s 2s/step - loss: 0.2956 - acc: 0.8745 - val_loss: 0.5881 - val_acc: 0.7730
Epoch 129/500
77/77 [==============================] - 187s 2s/step - loss: 0.2972 - acc: 0.8802 - val_loss: 0.8118 - val_acc: 0.7046
Epoch 130/500
77/77 [==============================] - 187s 2s/step - loss: 

...

77/77 [==============================] - 185s 2s/step - loss: 0.1610 - acc: 0.9371 - val_loss: 0.9392 - val_acc: 0.7493
Epoch 246/500
77/77 [==============================] - 185s 2s/step - loss: 0.1537 - acc: 0.9397 - val_loss: 0.8949 - val_acc: 0.7616
Epoch 247/500
77/77 [==============================] - 184s 2s/step - loss: 0.1614 - acc: 0.9355 - val_loss: 0.8198 - val_acc: 0.7849
Epoch 248/500
77/77 [==============================] - 186s 2s/step - loss: 0.1577 - acc: 0.9379 - val_loss: 0.7152 - val_acc: 0.8087
Epoch 249/500
77/77 [==============================] - 184s 2s/step - loss: 0.1567 - acc: 0.9390 - val_loss: 0.9135 - val_acc: 0.7603
Epoch 250/500
77/77 [==============================] - 187s 2s/step - loss: 0.1589 - acc: 0.9368 - val_loss: 0.7833 - val_acc: 0.7989
Epoch 251/500
77/77 [==============================] - 183s 2s/step - loss: 0.1597 - acc: 0.9373 - val_loss: 0.7993 - val_acc: 0.7915
Epoch 252/500
77/77 [==============================] - 186s 2s/step - loss: 

77/77 [==============================] - 185s 2s/step - loss: 0.1280 - acc: 0.9502 - val_loss: 0.8342 - val_acc: 0.7886
Epoch 307/500
77/77 [==============================] - 184s 2s/step - loss: 0.1249 - acc: 0.9521 - val_loss: 0.8423 - val_acc: 0.7849
Epoch 308/500
77/77 [==============================] - 189s 2s/step - loss: 0.1261 - acc: 0.9514 - val_loss: 0.7727 - val_acc: 0.7845
Epoch 309/500
77/77 [==============================] - 187s 2s/step - loss: 0.1239 - acc: 0.9522 - val_loss: 1.1706 - val_acc: 0.7349
Epoch 310/500
77/77 [==============================] - 187s 2s/step - loss: 0.1288 - acc: 0.9500 - val_loss: 1.2773 - val_acc: 0.7083
Epoch 311/500
77/77 [==============================] - 185s 2s/step - loss: 0.1246 - acc: 0.9524 - val_loss: 1.1713 - val_acc: 0.7263
Epoch 312/500
77/77 [==============================] - 188s 2s/step - loss: 0.1178 - acc: 0.9553 - val_loss: 0.9871 - val_acc: 0.7534
Epoch 313/500
77/77 [==============================] - 186s 2s/step - loss: 

77/77 [==============================] - 184s 2s/step - loss: 0.0987 - acc: 0.9617 - val_loss: 0.7724 - val_acc: 0.8136
Epoch 368/500
77/77 [==============================] - 187s 2s/step - loss: 0.0981 - acc: 0.9611 - val_loss: 0.8183 - val_acc: 0.8034
Epoch 369/500
77/77 [==============================] - 185s 2s/step - loss: 0.1010 - acc: 0.9615 - val_loss: 0.7832 - val_acc: 0.8132
Epoch 370/500
77/77 [==============================] - 187s 2s/step - loss: 0.0998 - acc: 0.9622 - val_loss: 0.9646 - val_acc: 0.8001
Epoch 371/500
77/77 [==============================] - 183s 2s/step - loss: 0.1016 - acc: 0.9610 - val_loss: 0.8084 - val_acc: 0.8152
Epoch 372/500
77/77 [==============================] - 189s 2s/step - loss: 0.0973 - acc: 0.9623 - val_loss: 0.8031 - val_acc: 0.8091
Epoch 373/500
77/77 [==============================] - 185s 2s/step - loss: 0.1024 - acc: 0.9603 - val_loss: 0.7687 - val_acc: 0.8169
Epoch 374/500
77/77 [==============================] - 187s 2s/step - loss: 

77/77 [==============================] - 187s 2s/step - loss: 0.0864 - acc: 0.9667 - val_loss: 1.2603 - val_acc: 0.7509
Epoch 429/500
77/77 [==============================] - 186s 2s/step - loss: 0.0842 - acc: 0.9669 - val_loss: 0.7874 - val_acc: 0.8226
Epoch 430/500
77/77 [==============================] - 186s 2s/step - loss: 0.0833 - acc: 0.9687 - val_loss: 0.8896 - val_acc: 0.8095
Epoch 431/500
77/77 [==============================] - 187s 2s/step - loss: 0.0829 - acc: 0.9686 - val_loss: 0.9958 - val_acc: 0.7890
Epoch 432/500
77/77 [==============================] - 187s 2s/step - loss: 0.0790 - acc: 0.9699 - val_loss: 0.9758 - val_acc: 0.7849
Epoch 433/500
77/77 [==============================] - 186s 2s/step - loss: 0.0870 - acc: 0.9663 - val_loss: 0.8184 - val_acc: 0.8185
Epoch 434/500
77/77 [==============================] - 186s 2s/step - loss: 0.0873 - acc: 0.9675 - val_loss: 0.8422 - val_acc: 0.8189
Epoch 435/500
77/77 [==============================] - 185s 2s/step - loss: 

77/77 [==============================] - 185s 2s/step - loss: 0.0703 - acc: 0.9743 - val_loss: 0.8280 - val_acc: 0.8152
Epoch 490/500
77/77 [==============================] - 185s 2s/step - loss: 0.0737 - acc: 0.9731 - val_loss: 0.9435 - val_acc: 0.7976
Epoch 491/500
77/77 [==============================] - 187s 2s/step - loss: 0.0740 - acc: 0.9725 - val_loss: 0.9369 - val_acc: 0.7898
Epoch 492/500
77/77 [==============================] - 187s 2s/step - loss: 0.0705 - acc: 0.9731 - val_loss: 0.9648 - val_acc: 0.8128
Epoch 493/500
77/77 [==============================] - 185s 2s/step - loss: 0.0780 - acc: 0.9700 - val_loss: 0.9435 - val_acc: 0.8095
Epoch 494/500
77/77 [==============================] - 185s 2s/step - loss: 0.0722 - acc: 0.9736 - val_loss: 1.2552 - val_acc: 0.7501
Epoch 495/500
77/77 [==============================] - 186s 2s/step - loss: 0.0755 - acc: 0.9702 - val_loss: 1.0037 - val_acc: 0.7948
Epoch 496/500
77/77 [==============================] - 186s 2s/step - loss: 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Acc')
plt.ylabel('acc')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
true_label = []
predicted_label = []
for index in test_indices:
    msrp = prices[index]
    true_label.append(str(msrp))
    
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    
    # Prediction outputs softmax vector
    prediction = new_model.predict(data)
    
    # Set most confident prediction as label, and convert it to our price scale
    label = np.argmax(prediction) * 25 + 25
    predicted_label.append(str(label))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print("Classification report:\n%s\n"
      % (classification_report(true_label, predicted_label)))
print("Confusion matrix:\n%s" % confusion_matrix(true_label, predicted_label))
